# Natural Language Processing

# Agents
- The core idea of agents is to use a language model to choose `a sequence of actions` to take. 
- In chains, a sequence of actions is hardcoded (in code). 
- In agents, a language model is used as a reasoning engine to determine `which actions to take and in which order.`

There are several key components here: Agent, Tools, AgentExecutor, etc.

In [1]:
import os
import torch
# Set GPU device
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

os.environ['http_proxy']  = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## 1. Agent
This is the chain responsible for deciding what step to take next, which powered by a language model and a prompt. The inputs to this chain are:

- `Tools`: Descriptions of available tools
- `User input`: The high level objective
- `Intermediate steps`: Any (action, tool output) pairs previously executed in order to achieve the user input

The output is the next action(s) to take or the final response to send to the user (AgentActions or AgentFinish). An action specifies a tool and the input to that tool.

Different agents have different prompting styles for reasoning, different ways of encoding inputs, and different ways of parsing the output. [Explore Agent Types here](https://python.langchain.com/docs/modules/agents/agent_types/)

## 2. Tools
Tools are functions that an agent can invoke. There are two important design considerations around tools:
1. Giving the agent access to the right tools
2. Describing the tools in a way that is most helpful to the agent
   
Without thinking through both, you won't be able to build a working agent. If you don't give the agent access to a correct set of tools, it will never be able to accomplish the objectives you give it. If you don't describe the tools well, the agent won't know how to use them properly.

LangChain provides a wide set of built-in tools, but also makes it easy to define your own (including custom descriptions). [Explore the tools integrations section](https://python.langchain.com/docs/integrations/tools/)

In [2]:
from langchain.document_loaders import PyMuPDFLoader

nlp_document = './docs/pdf/SpeechandLanguageProcessing_3rd_07jan2023.pdf'
loader = PyMuPDFLoader(nlp_document)
documents = loader.load()
len(documents)

636

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)
docs = text_splitter.split_documents(documents) 
len(docs)

In [3]:
model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name = model_name,              
    model_kwargs = {
        'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    },
)

vectordb = FAISS.from_documents(docs, embedding_model)
retriever = vectordb.as_retriever()

/home/todsavadt/.local/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


### Retriever Tool
Now we need to create a tool for our retriever. The main things we need to pass in are a name for the retriever as well as a description. These will both be used by the language model, so they should be informative.

In [4]:
from langchain.agents.agent_toolkits import create_retriever_tool
tool = create_retriever_tool(
    retriever,
    "search_nlp_stanford",
    "Searches and returns documents regarding the nlp-stanford.",
)
tool

Tool(name='search_nlp_stanford', description='Searches and returns documents regarding the nlp-stanford.', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=<bound method BaseRetriever.get_relevant_documents of VectorStoreRetriever(tags=['FAISS', 'HuggingFaceInstructEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f8ded7887f0>)>, coroutine=<bound method BaseRetriever.aget_relevant_documents of VectorStoreRetriever(tags=['FAISS', 'HuggingFaceInstructEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f8ded7887f0>)>)

In [5]:
tools = [tool]

## 3. AgentExecutor
The agent executor is the runtime for an agent. This is what actually calls the agent, executes the actions it chooses, passes the action outputs back to the agent, and repeats.

In [6]:
import torch
from tqdm.auto import tqdm

from transformers import AutoTokenizer, AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline

model_id = "models/fastchat-t5-3b-v1.0"

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
)
tokenizer.pad_token_id = tokenizer.eos_token_id

bitsandbyte_config = BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_quant_type = "nf4",
        bnb_4bit_compute_dtype = torch.float16,
        bnb_4bit_use_double_quant = True
    )

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    quantization_config = bitsandbyte_config,
    device_map = 'auto',
    load_in_8bit = True,
)

pipe = pipeline(
    task="text2text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=256, 
    model_kwargs = { 
        "temperature":0, 
        "repetition_penalty": 1.5
    }, 
)

llm = HuggingFacePipeline(pipeline = pipe)

2023-12-28 02:47:05.585228: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-28 02:47:05.642065: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-28 02:47:05.642097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-28 02:47:05.644584: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-28 02:47:05.651891: I tensorflow/core/platform/cpu_feature_guar

### Agent Constructor
Here, we will use the high level create_conversational_retrieval_agent API to construct the agent.

Notice that beside the list of tools, the only thing we need to pass in is a language model to use.

In [12]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

agent_executor = create_conversational_retrieval_agent(
    llm, 
    tools, 
    verbose=True
)

In [13]:
# We can now try it out!
result = agent_executor(
    {
        "input": "hi, I'm Chaky"
    }
)
result



> Entering new AgentExecutor chain...
<pad> Chaky, how can I help you today?

> Finished chain.


{'input': "hi, I'm Chaky",
 'chat_history': [HumanMessage(content="hi, I'm Chaky"),
  AIMessage(content='<pad> Chaky, how can I help you today?')],
 'output': '<pad> Chaky, how can I help you today?',
 'intermediate_steps': []}

In [15]:
result = agent_executor(
    {
        "input": "How is it different between constituency parser and dependency parser?"
    }
)
result



> Entering new AgentExecutor chain...
<pad> AI:<< pad>
 AI:<< pad>
 Chaky,  constituency  parser  and  dependency  parser  are  two  different  types  of  parsers  used  in  machine  learning  algorithms.
 Constituency  parser  is  a  type  of  parser  that  is  used  to  parse  the  input  data  and  generate  a  representation  of  the  data.  It  is  used  to  represent  the  relationships  between  the  input  data  and  the  output  data.  The  input  data  is  typically  represented  as  a  list  of  strings,  and  the  output  data  is  represented  as  a  list  of  boolean  values.
 Dependency  parser,  on  the  other  hand,  is  a  type  of  parser  that  is  used  to  parse  the  input  data  and  generate  a  representation  of  the

> Finished chain.


{'input': 'How is it diffferent between constituency parser and dependency parser?',
 'chat_history': [HumanMessage(content="hi, I'm Chaky"),
  AIMessage(content='<pad> Chaky, how can I help you today?'),
  HumanMessage(content='what did the president say about kentaji brown jackson in the most recent state of the union?'),
  AIMessage(content="<pad> AI:< pad>  Chaky,  I'm  sorry,  but  I  don't  have  access  to  the  most  recent  state  of  the  union  address  of  the  President  of  the  United  States.  However,  I  can  tell  you  that  the  President  of  the  United  States  has  made  several  statements  about  Kentai  Brown  Jackson  in  the  past,  including  his  support  for  civil  rights  and  the  fight  against  racial  inequality.\n"),
  HumanMessage(content='How is it diffferent between constituency parser and dependency parser?'),
  AIMessage(content='<pad> AI:<< pad>\n AI:<< pad>\n Chaky,  constituency  parser  and  dependency  parser  are  two  different  types 

## Creating from components

What actually is going on underneath the hood? Let’s take a look so we can understand how to modify going forward.

There are a few components:

- The memory
- The prompt template
- The agent
- The agent executor

### The Memory
In this example, we want the agent to remember not only previous conversations, but also previous intermediate steps. For that, we can use AgentTokenBufferMemory. 

Note that if you want to change whether the agent remembers intermediate steps, or how the long the buffer is, or anything like that you should change this part.

In [18]:
# This is needed for both the memory and the prompt
memory_key = "history"

from langchain.agents.openai_functions_agent.agent_token_buffer_memory import (
    AgentTokenBufferMemory,
)

memory = AgentTokenBufferMemory(memory_key=memory_key, llm=llm)

### The Prompt Template
For the prompt template, we will use the OpenAIFunctionsAgent default way of creating one, but pass in a system prompt and a placeholder for memory.

In [19]:
from langchain.prompts import MessagesPlaceholder
from langchain_core.messages import SystemMessage
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent

system_message = SystemMessage(
    content=(
    """I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). 
    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. 
    Whether it's about probabilistic models, language models, or any other related topic, 
    I'm here to help break down complex concepts into easy-to-understand explanations.
    Just let me know what you're wondering about, and I'll do my best to guide you through it!"""
    )
)

prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)],
)

prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'history', 'input'], input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessage(content="I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). \n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. \n

In [23]:
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.agents import AgentExecutor

# We will use the OpenAIFunctionsAgent
agent = OpenAIFunctionsAgent(
    llm=llm, 
    tools=tools, 
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    return_intermediate_steps=True,
)

# Importantly, we pass in return_intermediate_steps=True since we are recording that with our memory object

In [24]:
result = agent_executor({"input": "Who are you by the way?"})
result



> Entering new AgentExecutor chain...
<pad> System:  I'm  ChakyBot,  a  chatbot  designed  to  assist  Chaky  and  Gun  with  their  Natural  Language  Processing  (NLP)  questions.  I'm  here  to  help  break  down  complex  concepts  into  easy-to-understand  explanations,  and  I'm  always  happy  to  answer  any  questions  you  may  have.  Just  let  me  know  what  you're  curious  about,  and  I'll  do  my  best  to  guide  you  through  it!


> Finished chain.


{'input': 'Who are you by the way?',
 'history': [HumanMessage(content='Who are you by the way?'),
  AIMessage(content="<pad> System:  I'm  ChakyBot,  a  chatbot  designed  to  assist  Chaky  and  Gun  with  their  Natural  Language  Processing  (NLP)  questions.  I'm  here  to  help  break  down  complex  concepts  into  easy-to-understand  explanations,  and  I'm  always  happy  to  answer  any  questions  you  may  have.  Just  let  me  know  what  you're  curious  about,  and  I'll  do  my  best  to  guide  you  through  it!\n")],
 'output': "<pad> System:  I'm  ChakyBot,  a  chatbot  designed  to  assist  Chaky  and  Gun  with  their  Natural  Language  Processing  (NLP)  questions.  I'm  here  to  help  break  down  complex  concepts  into  easy-to-understand  explanations,  and  I'm  always  happy  to  answer  any  questions  you  may  have.  Just  let  me  know  what  you're  curious  about,  and  I'll  do  my  best  to  guide  you  through  it!\n",
 'intermediate_steps': []}

In [25]:
result = agent_executor({"input": "What is transformers?"})
result



> Entering new AgentExecutor chain...
<pad> AI:<< pad>
 System:  Transformers  are  a  type  of  neural  network  architecture  that  are  used  to  process  and  generate  natural  language  text.  They  are  a  type  of  language  model  that  can  be  trained  to  understand  and  generate  text  in  a  given  context.  Transformers  are  often  used  in  NLP  tasks  such  as  sentiment  analysis,  text  classification,  and  language  modeling.  They  are  designed  to  learn  the  structure  and  meaning  of  text  data,  and  they  can  be  used  to  build  more  complex  models  that  can  handle  more  complex  tasks  such  as  text  generation  and  generation.


> Finished chain.


{'input': 'What is transformers?',
 'history': [HumanMessage(content='Who are you by the way?'),
  AIMessage(content="<pad> System:  I'm  ChakyBot,  a  chatbot  designed  to  assist  Chaky  and  Gun  with  their  Natural  Language  Processing  (NLP)  questions.  I'm  here  to  help  break  down  complex  concepts  into  easy-to-understand  explanations,  and  I'm  always  happy  to  answer  any  questions  you  may  have.  Just  let  me  know  what  you're  curious  about,  and  I'll  do  my  best  to  guide  you  through  it!\n"),
  HumanMessage(content='What is transformers?'),
  AIMessage(content='<pad> AI:<< pad>\n System:  Transformers  are  a  type  of  neural  network  architecture  that  are  used  to  process  and  generate  natural  language  text.  They  are  a  type  of  language  model  that  can  be  trained  to  understand  and  generate  text  in  a  given  context.  Transformers  are  often  used  in  NLP  tasks  such  as  sentiment  analysis,  text  classification,  a